<a href="https://colab.research.google.com/github/mvm1964/MGVB/blob/main/ann_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
# prompt: create a torch neural net class with two hided layers and a single continuous node in the output layer

class NeuralNetwork(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size1),
            nn.ReLU(),
            nn.Linear(hidden_size1, hidden_size2),
            nn.ReLU(),
            nn.Linear(hidden_size2, 1) # Single continuous output node
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


In [3]:
# prompt: create pandas dataframe with 20 continuous columns and 1000 rows filled with random numbers from 0 to 1

import pandas as pd
import numpy as np

# Create a DataFrame with 20 continuous columns and 1000 rows
data = np.random.rand(1000, 20)
df = pd.DataFrame(data)

df


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0,0.734096,0.602271,0.467170,0.334864,0.546003,0.093872,0.108012,0.910067,0.716150,0.431305,0.647608,0.721674,0.932800,0.539230,0.811333,0.780170,0.619829,0.468551,0.277438,0.865233
1,0.200860,0.802319,0.205490,0.066150,0.252995,0.381023,0.786644,0.015511,0.971273,0.974456,0.973005,0.111705,0.388496,0.810328,0.205987,0.870840,0.528169,0.584040,0.923397,0.052451
2,0.571390,0.644237,0.945437,0.197144,0.838148,0.414760,0.854055,0.019166,0.805045,0.478645,0.887938,0.465142,0.311852,0.923975,0.696313,0.552951,0.875093,0.261781,0.494256,0.118756
3,0.701937,0.225290,0.802527,0.472299,0.613060,0.894203,0.553827,0.121115,0.521391,0.592925,0.036307,0.112650,0.609620,0.383527,0.784043,0.418580,0.320791,0.953224,0.621376,0.891307
4,0.279881,0.907052,0.888910,0.620954,0.936808,0.164742,0.505981,0.694664,0.875181,0.009884,0.971700,0.736000,0.421143,0.811618,0.620982,0.012419,0.329632,0.502020,0.804090,0.968603
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0.618106,0.633513,0.746145,0.942642,0.519302,0.150337,0.185497,0.741242,0.473998,0.847516,0.899622,0.753888,0.107088,0.467572,0.812101,0.206163,0.322107,0.044631,0.380378,0.356163
996,0.553406,0.751374,0.592898,0.983367,0.127327,0.631051,0.022072,0.119352,0.727361,0.023173,0.567663,0.861446,0.467905,0.510059,0.656915,0.939231,0.548823,0.869460,0.292915,0.456908
997,0.182276,0.612224,0.880216,0.055005,0.695716,0.616120,0.854974,0.206773,0.383311,0.191294,0.018999,0.851160,0.828219,0.782314,0.095799,0.102094,0.790740,0.263034,0.508107,0.109070
998,0.657488,0.312623,0.658516,0.866386,0.313397,0.633716,0.403737,0.712325,0.934357,0.718592,0.241048,0.366087,0.861216,0.579708,0.735835,0.675598,0.437343,0.168821,0.630509,0.860670


In [19]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#device = torch.device("cpu")
print(f"Using {device} device")

Using cuda device


In [20]:
model = NeuralNetwork(input_size=19, hidden_size1=10, hidden_size2=10).to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=19, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): ReLU()
    (4): Linear(in_features=10, out_features=1, bias=True)
  )
)


In [21]:
X = torch.rand(1, 19, device=device)
model(X)

tensor([[0.0919]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [22]:
# Read data, convert to NumPy arrays
X = df.iloc[:, 1:20].values
y = df.iloc[:, 0].values

#device = torch.device("cpu")

# convert into PyTorch tensors
X = torch.tensor(X, dtype=torch.float32, device=device)
y = torch.tensor(y, dtype=torch.float32, device=device).reshape(-1, 1)

# create DataLoader, then take one batch
loader = DataLoader(list(zip(X,y)), shuffle=True, batch_size=16)
for X_batch, y_batch in loader:
    print(X_batch, y_batch)
    break


tensor([[0.4052, 0.5181, 0.8507, 0.3696, 0.8388, 0.3964, 0.9876, 0.1692, 0.5965,
         0.5744, 0.5686, 0.7973, 0.5024, 0.7546, 0.4155, 0.8234, 0.8630, 0.6592,
         0.3834],
        [0.0880, 0.0223, 0.2870, 0.4906, 0.9775, 0.5594, 0.9564, 0.0120, 0.1629,
         0.0327, 0.2706, 0.7516, 0.5820, 0.1913, 0.6010, 0.5365, 0.0169, 0.7857,
         0.9257],
        [0.8314, 0.9761, 0.5765, 0.7251, 0.4561, 0.8702, 0.4481, 0.2610, 0.1450,
         0.4203, 0.0072, 0.5184, 0.1144, 0.7832, 0.1188, 0.4135, 0.2966, 0.7936,
         0.6391],
        [0.4847, 0.6962, 0.6071, 0.0496, 0.5525, 0.9517, 0.3792, 0.2877, 0.0536,
         0.2377, 0.7511, 0.6196, 0.5723, 0.8283, 0.5345, 0.9885, 0.3039, 0.4969,
         0.8826],
        [0.1140, 0.0535, 0.6783, 0.5405, 0.9096, 0.1712, 0.5657, 0.4206, 0.8215,
         0.4641, 0.2365, 0.3849, 0.0594, 0.3402, 0.8667, 0.3322, 0.6787, 0.1707,
         0.6562],
        [0.5385, 0.5805, 0.5430, 0.7290, 0.6647, 0.3920, 0.4791, 0.1280, 0.9238,
         0.3412, 0.

In [23]:
import torch.optim as optim
from sklearn.model_selection import train_test_split
# train-test split for evaluation of the model
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, shuffle=True)

# set up DataLoader for training set
loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)


In [24]:
# train
n_epochs = 200
loss_fn = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)
model.train()
for epoch in range(n_epochs):
    for X_batch, y_batch in loader:
        y_pred = model(X_batch)
        loss = loss_fn(y_pred, y_batch)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# evaluate accuracy after training
model.eval()
y_pred = model(X_test)
acc = (y_pred.round() == y_test).float().mean()
acc = float(acc)
print("Model accuracy: %.2f%%" % (acc*100))

Model accuracy: 0.00%
